In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
dataset=pd.read_csv('dataset/movies.csv')
# dataset=dataset.dropna(axis='rows')
dataset.drop(columns=dataset.columns[-1],  axis=1,  inplace=True)
dataset=dataset.iloc[:6000,:]
dataset.shape

/tmp/ipykernel_27101/4237089710.py:1: DtypeWarning: Columns (3,5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset=pd.read_csv('dataset/movies.csv')


(6000, 8)

In [4]:
dataset.columns=["title",'year',"overview","ratings","genres","popularity","cast",'poster']

In [5]:
dataset['id']=[i for i in range(dataset.shape[0])]
dataset['year']=dataset['year'].astype("string")
# len(dataset['year'][2])
dataset.shape[0]

dataset=dataset.dropna()
dataset['year'].isnull().sum()
dataset.isnull().sum()

title         0
year          0
overview      0
ratings       0
genres        0
popularity    0
cast          0
poster        0
id            0
dtype: int64

In [6]:
dataset=dataset[["id","title","year","overview","ratings","genres","popularity","cast","poster"]]
dataset.drop_duplicates()

,id,title,year,overview,ratings,genres,popularity,cast,poster
0,0,The Walking Dead,2010–2022,\nSheriff Deputy Rick Grimes wakes up from a c...,8.1,"\nDrama, Horror, Thriller",999787,"Andrew Lincoln,Norman Reedus,Melissa McBride,L...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
1,1,Jack Ryan,2018–,"\nUp-and-coming CIA analyst, Jack Ryan, is thr...",8,"\nAction, Drama, Thriller",138760,"John Krasinski,Wendell Pierce,Michael Kelly,Jo...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
2,2,Knives Out,2019,\nA detective investigates the death of the pa...,7.9,"\nComedy, Crime, Drama",687013,"Rian Johnson,Daniel Craig,Chris Evans,Ana de A...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
3,3,Game of Thrones,2011–2019,\nNine noble families fight for control over t...,9.2,"\nAction, Adventure, Drama",2110375,"Emilia Clarke,Peter Dinklage,Kit Harington,Len...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
4,4,Avatar,2009,\nA paraplegic Marine dispatched to the moon P...,7.9,"\nAction, Adventure, Fantasy",1306723,"James Cameron,Sam Worthington,Zoe Saldana,Sigo...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
...,...,...,...,...,...,...,...,...,...
5995,5995,Ghajini,2008,\nA short-term memory loss patient sets out on...,7.3,"\nAction, Drama, Mystery",62832,"A.R. Murugadoss,Aamir Khan,Asin Thottumkal,Jia...",https://m.media-amazon.com/images/M/MV5BMzEzMj...
5996,5996,Lasciati andare,2017,\nA psychoanalyst named Elia goes to the gym a...,6,\nComedy,1277,"Francesco Amato,Toni Servillo,Verónica Echegui...",https://m.media-amazon.com/images/M/MV5BMzEzMj...
5997,5997,Tim and Eric's Billion Dollar Movie,2012,\nTwo guys get a billion dollars to make a mov...,5.3,\nComedy,10274,"Tim Heidecker,Eric Wareheim,Tim Heidecker,Eric...",https://m.media-amazon.com/images/M/MV5BMzEzMj...
5998,5998,Gullak,2019–,\nSet in quaint by-lanes in the heart of India...,9.1,"\nComedy, Drama, Family",17780,"Jameel Khan,Geetanjali Kulkarni,Vaibhav Raj Gu...",https://m.media-amazon.com/images/M/MV5BMzEzMj...


In [7]:
dataset['year'] = dataset['year'].str.replace('–','')
dataset['year'].astype('string')



0       20102022
1          2018 
2           2019
3       20112019
4           2009
          ...   
5995        2008
5996        2017
5997        2012
5998       2019 
5999        2013
Name: year, Length: 5999, dtype: string

In [8]:
dataset.loc[dataset['year'].str.len() == 8, 'year'] = dataset.loc[dataset['year'].str.len() == 8, 'year'].str.slice(4,8)

In [9]:
dataset['overview']=dataset['overview'].str.replace('\n','')
dataset['genres']=dataset['genres'].str.replace('\n','')
dataset

,id,title,year,overview,ratings,genres,popularity,cast,poster
0,0,The Walking Dead,2022,Sheriff Deputy Rick Grimes wakes up from a com...,8.1,"Drama, Horror, Thriller",999787,"Andrew Lincoln,Norman Reedus,Melissa McBride,L...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
1,1,Jack Ryan,2018,"Up-and-coming CIA analyst, Jack Ryan, is thrus...",8,"Action, Drama, Thriller",138760,"John Krasinski,Wendell Pierce,Michael Kelly,Jo...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
2,2,Knives Out,2019,A detective investigates the death of the patr...,7.9,"Comedy, Crime, Drama",687013,"Rian Johnson,Daniel Craig,Chris Evans,Ana de A...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
3,3,Game of Thrones,2019,Nine noble families fight for control over the...,9.2,"Action, Adventure, Drama",2110375,"Emilia Clarke,Peter Dinklage,Kit Harington,Len...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
4,4,Avatar,2009,A paraplegic Marine dispatched to the moon Pan...,7.9,"Action, Adventure, Fantasy",1306723,"James Cameron,Sam Worthington,Zoe Saldana,Sigo...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
...,...,...,...,...,...,...,...,...,...
5995,5995,Ghajini,2008,A short-term memory loss patient sets out on h...,7.3,"Action, Drama, Mystery",62832,"A.R. Murugadoss,Aamir Khan,Asin Thottumkal,Jia...",https://m.media-amazon.com/images/M/MV5BMzEzMj...
5996,5996,Lasciati andare,2017,A psychoanalyst named Elia goes to the gym and...,6,Comedy,1277,"Francesco Amato,Toni Servillo,Verónica Echegui...",https://m.media-amazon.com/images/M/MV5BMzEzMj...
5997,5997,Tim and Eric's Billion Dollar Movie,2012,Two guys get a billion dollars to make a movie...,5.3,Comedy,10274,"Tim Heidecker,Eric Wareheim,Tim Heidecker,Eric...",https://m.media-amazon.com/images/M/MV5BMzEzMj...
5998,5998,Gullak,2019,"Set in quaint by-lanes in the heart of India, ...",9.1,"Comedy, Drama, Family",17780,"Jameel Khan,Geetanjali Kulkarni,Vaibhav Raj Gu...",https://m.media-amazon.com/images/M/MV5BMzEzMj...


In [10]:
dataset['genres']=dataset['genres'].astype(dtype='string')
dataset['genres'] = dataset['genres'].apply(lambda x: x.split(", "))


In [11]:
dataset['genres'].replace(',',' ')

0          [Drama, Horror, Thriller            ]
1          [Action, Drama, Thriller            ]
2             [Comedy, Crime, Drama            ]
3         [Action, Adventure, Drama            ]
4       [Action, Adventure, Fantasy            ]
                          ...                   
5995        [Action, Drama, Mystery            ]
5996                        [Comedy            ]
5997                        [Comedy            ]
5998         [Comedy, Drama, Family            ]
5999                        [Comedy            ]
Name: genres, Length: 5999, dtype: object

In [12]:
dataset['title'] = dataset['title'].apply(lambda x: x.split(", "))
dataset['overview'] = dataset['overview'].apply(lambda x:x.split(", "))
# dataset['overview'] = dataset['overview'].str.replace('-','')
dataset['cast'] = dataset['cast'].str.replace('-','')
# dataset['genres'] = dataset['genres'].str.replace(',','')
dataset['cast'] = dataset['cast'].apply(lambda x:x.split(", "))

dataset

,id,title,year,overview,ratings,genres,popularity,cast,poster
0,0,[The Walking Dead],2022,[Sheriff Deputy Rick Grimes wakes up from a co...,8.1,"[Drama, Horror, Thriller ]",999787,"[Andrew Lincoln,Norman Reedus,Melissa McBride,...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
1,1,[Jack Ryan],2018,"[Up-and-coming CIA analyst, Jack Ryan, is thru...",8,"[Action, Drama, Thriller ]",138760,"[John Krasinski,Wendell Pierce,Michael Kelly,J...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
2,2,[Knives Out],2019,[A detective investigates the death of the pat...,7.9,"[Comedy, Crime, Drama ]",687013,"[Rian Johnson,Daniel Craig,Chris Evans,Ana de ...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
3,3,[Game of Thrones],2019,[Nine noble families fight for control over th...,9.2,"[Action, Adventure, Drama ]",2110375,"[Emilia Clarke,Peter Dinklage,Kit Harington,Le...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
4,4,[Avatar],2009,[A paraplegic Marine dispatched to the moon Pa...,7.9,"[Action, Adventure, Fantasy ]",1306723,"[James Cameron,Sam Worthington,Zoe Saldana,Sig...",https://m.media-amazon.com/images/M/MV5BOGFiZj...
...,...,...,...,...,...,...,...,...,...
5995,5995,[Ghajini],2008,[A short-term memory loss patient sets out on ...,7.3,"[Action, Drama, Mystery ]",62832,"[A.R. Murugadoss,Aamir Khan,Asin Thottumkal,Ji...",https://m.media-amazon.com/images/M/MV5BMzEzMj...
5996,5996,[Lasciati andare],2017,[A psychoanalyst named Elia goes to the gym an...,6,[Comedy ],1277,"[Francesco Amato,Toni Servillo,Verónica Echegu...",https://m.media-amazon.com/images/M/MV5BMzEzMj...
5997,5997,[Tim and Eric's Billion Dollar Movie],2012,[Two guys get a billion dollars to make a movi...,5.3,[Comedy ],10274,"[Tim Heidecker,Eric Wareheim,Tim Heidecker,Eri...",https://m.media-amazon.com/images/M/MV5BMzEzMj...
5998,5998,[Gullak],2019,"[Set in quaint by-lanes in the heart of India,...",9.1,"[Comedy, Drama, Family ]",17780,"[Jameel Khan,Geetanjali Kulkarni,Vaibhav Raj G...",https://m.media-amazon.com/images/M/MV5BMzEzMj...


In [13]:
[[dataset['title']]]
# dataset['overview']=list(dataset['overview'])

[[0                          [The Walking Dead]
  1                                 [Jack Ryan]
  2                                [Knives Out]
  3                           [Game of Thrones]
  4                                    [Avatar]
                          ...                  
  5995                                [Ghajini]
  5996                        [Lasciati andare]
  5997    [Tim and Eric's Billion Dollar Movie]
  5998                                 [Gullak]
  5999                   [InAPPropriate Comedy]
  Name: title, Length: 5999, dtype: object]]

In [14]:
dataset['tags'] = dataset['title'] + dataset['overview'] + dataset['genres'] +  dataset['cast'] 
dataset['tags'][0]

['The Walking Dead',
 'Sheriff Deputy Rick Grimes wakes up from a coma to learn the world is in ruins and must lead a group of survivors to stay alive.',
 'Drama',
 'Horror',
 'Thriller            ',
 'Andrew Lincoln,Norman Reedus,Melissa McBride,Lauren Cohan,']

In [15]:
# dataset['tags'] = dataset['tags'].apply(lambda x:"".join(x))
dataset1= dataset[['id','title','year','overview','tags','genres']]
# dataset1['tags'] = dataset1['tags'].str.replace('-',' ')
dataset1.head(1)

,id,title,year,overview,tags,genres
0,0,[The Walking Dead],2022,[Sheriff Deputy Rick Grimes wakes up from a co...,"[The Walking Dead, Sheriff Deputy Rick Grimes ...","[Drama, Horror, Thriller ]"


In [16]:
dataset1['tags'] = dataset1['tags'].apply(lambda x:"".join(x))
dataset1['title'] = dataset1['title'].apply(lambda x: "".join(x))
dataset1['genres']=dataset1['genres'].replace(',',' ')
dataset1['overview'] = dataset1['overview'].apply(lambda x: "".join(x))
dataset1['genres'] = dataset1['genres'].apply(lambda x: " ".join(x))
# dataset['genres'] = dataset['genres'].apply(lambda x: x.strip(", "))
dataset1

/tmp/ipykernel_27101/557954354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset1['tags'] = dataset1['tags'].apply(lambda x:"".join(x))
/tmp/ipykernel_27101/557954354.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset1['title'] = dataset1['title'].apply(lambda x: "".join(x))
/tmp/ipykernel_27101/557954354.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,id,title,year,overview,tags,genres
0,0,The Walking Dead,2022,Sheriff Deputy Rick Grimes wakes up from a com...,The Walking DeadSheriff Deputy Rick Grimes wak...,Drama Horror Thriller
1,1,Jack Ryan,2018,Up-and-coming CIA analystJack Ryanis thrust in...,Jack RyanUp-and-coming CIA analystJack Ryanis ...,Action Drama Thriller
2,2,Knives Out,2019,A detective investigates the death of the patr...,Knives OutA detective investigates the death o...,Comedy Crime Drama
3,3,Game of Thrones,2019,Nine noble families fight for control over the...,Game of ThronesNine noble families fight for c...,Action Adventure Drama
4,4,Avatar,2009,A paraplegic Marine dispatched to the moon Pan...,AvatarA paraplegic Marine dispatched to the mo...,Action Adventure Fantasy
...,...,...,...,...,...,...
5995,5995,Ghajini,2008,A short-term memory loss patient sets out on h...,GhajiniA short-term memory loss patient sets o...,Action Drama Mystery
5996,5996,Lasciati andare,2017,A psychoanalyst named Elia goes to the gym and...,Lasciati andareA psychoanalyst named Elia goes...,Comedy
5997,5997,Tim and Eric's Billion Dollar Movie,2012,Two guys get a billion dollars to make a movie...,Tim and Eric's Billion Dollar MovieTwo guys ge...,Comedy
5998,5998,Gullak,2019,Set in quaint by-lanes in the heart of IndiaGu...,GullakSet in quaint by-lanes in the heart of I...,Comedy Drama Family


In [17]:
dataset1['tags'] = dataset1['tags'].apply(lambda x:x.lower())
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

/tmp/ipykernel_27101/541602581.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset1['tags'] = dataset1['tags'].apply(lambda x:x.lower())


In [18]:
# dataset1['tags']=dataset1['tags'].replace('-',' ')
# dataset1['tags']=dataset1['tags'].replace('-',' ')
# for i in range(dataset1.shape[0]):
#     dataset1['tags'][i]=dataset1['tags'][i].replace('-',' ')
#     dataset1['tags'][i]=dataset1['tags'][i].replace(',',' ')
    
# dataset1

In [19]:
# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [20]:
dataset1['tags'] = dataset1['tags'].apply(stem)

/tmp/ipykernel_27101/1315034211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset1['tags'] = dataset1['tags'].apply(stem)


In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
cv = CountVectorizer(max_features=5000, stop_words='english')

vectors = cv.fit_transform(dataset1['tags']).toarray()
vectors_genre=cv.fit_transform(dataset1['genres'])
vectors_year=cv.fit_transform(dataset1['year'])


similarity = cosine_similarity(vectors)
similarity_genre=cosine_similarity(vectors_genre)
similarity_year=cosine_similarity(vectors_year)


In [22]:
import pickle
filename = 'countvectorizer.pkl'
pickle.dump(cv, open(filename, 'wb'))

filename = 'vectorizer_genre.pkl'
pickle.dump(vectors_genre, open(filename, 'wb'))


filename = 'vectorizer_recommend.pkl'
pickle.dump(vectors, open(filename, 'wb'))


filename = 'similarity_recommend.pkl'
pickle.dump(similarity, open(filename, 'wb'))

filename = 'similarity_genre.pkl'
pickle.dump(similarity_genre, open(filename, 'wb'))


In [23]:
dataset['title'] = dataset['title'].apply(lambda x: " ".join(x))
def recommend(movie,n_rec=8):
    movie_index = dataset1.index[dataset1['title'] == movie].index[0]
    # if movie_index:
        # movie_index = movie_index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[0:n_rec]
    for i in movies_list:
            print(dataset1.iloc[i[0]].title)
    else:
        print("Movie not found in the dataset")
def genre_recommendation(genres, n_rec=10):
    genres = genres[:4]
    vectors_genre=cv.fit_transform(dataset1['genres'])
    similarity_genre=cosine_similarity(vectors_genre)
    movie_indices = [i for i, gen in enumerate(dataset1['genres']) if all(g in gen for g in genres)]
    genre_similarity = similarity_genre[movie_indices].mean(axis=0)
    genre_similarity = genre_similarity.argsort()[::-1]
    for i in genre_similarity[:n_rec]:
        print(dataset1.iloc[i].title)
        print(dataset1.iloc[i].overview)
        print(dataset1.iloc[i].year)

# def year_recommendation(year, n_rec):
#     movie_indices = [i for i, yr in enumerate(dataset1['year']) if all(g in yr for g in year)]
#     year_similarity = similarity_genre[movie_indices].mean(axis=0)
#     year_similarity = year_similarity.argsort()[::-1]
#     for i in year_similarity[:n_rec]:
#         print(dataset1.iloc[i].title)
#         print(dataset1.iloc[i].overview)
#         print(dataset1.iloc[i].year)


In [ ]:
recommend("Westworld")

In [25]:
genre_recommendation(["Sci-Fi"],4)

The Matrix Reloaded
Freedom fighters NeoTrinity and Morpheus continue to lead the revolt against the Machine Armyunleashing their arsenal of extraordinary skills and weaponry against the systematic forces of repression and exploitation.
2003
The Matrix Revolutions
The human city of Zion defends itself against the massive invasion of the machines as Neo fights to end the war at another front while also opposing the rogue Agent Smith.
2003
Hulk
Bruce Bannera genetics researcher with a tragic pastsuffers an accident that causes him to transform into a raging green monster when he gets angry.
2003
X-Men Origins: Wolverine
The early years of James Loganfeaturing his rivalry with his brother Victor Creedhis service in the special forces team Weapon Xand his experimentation into the metal-lined mutant Wolverine.
2009


In [26]:
pickle.dump(recommend,open('recommend.pkl','wb'))
pickle.dump(genre_recommendation,open('genre_recommendation.pkl','wb'))
pickle.dump(dataset1,open('movie_list.pkl','wb'))